In [ ]:
firstrun = True

if firstrun != False:
    !pip install fredapi
    !pip install matplotlib
    !pip install plotly
    !pip install keras

    from fredapi import Fred
    fred = Fred(api_key='63769be4fe08f91aef948b301151587d')

    %matplotlib inline
    import matplotlib.pyplot as plt
    import pandas as pd
    import numpy as np
    import yfinance as yf
    import datetime
    from sklearn.preprocessing import MinMaxScaler
    from keras.models import Sequential, load_model
    from keras.layers import LSTM, Dense, Dropout
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    from sklearn.model_selection import train_test_split

def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

def getData(ticker_symbol, startday, endday):
    stock_data = yf.download(ticker_symbol, start=startday, end=endday)
    stock_data['Daily_Return'] = stock_data['Adj Close'].pct_change() * 100

    CPI = fred.get_series('MEDCPIM158SFRBCLE', observation_start=startday)
    M30US = fred.get_series('MORTGAGE30US', observation_start=startday)
    UNRA = fred.get_series('UNRATE', observation_start=startday)
    SP500 = fred.get_series('SP500', observation_start=startday)
    NEWH = fred.get_series('HOUST', observation_start=startday)
    NASDAQ = fred.get_series('NASDAQCOM', observation_start=startday)
    CARSA = fred.get_series('TOTALSA', observation_start=startday)

    df_fred = pd.concat([SP500, CPI, M30US, UNRA, NEWH, NASDAQ, CARSA], axis=1)
    data_names = ['SP500', 'CPI', 'M30US', 'UNRA', 'NEWH', 'NASDAQ', 'CARSA']
    df_fred.columns = data_names

    df_fred = df_fred.ffill().fillna(method='backfill')
    stock_data = stock_data.ffill().fillna(method='backfill')
    df = pd.concat((stock_data, df_fred), axis=1)
    df.dropna(inplace=True)
    df.to_csv('data.csv', index=False)

    return df

def predLSTM(df, target, dayspan, seq_length):
    df_tail = df.tail(dayspan)
    df = df.iloc[:-dayspan].reset_index(drop=True)

    target_col = target
    features = df.columns
    X = df[features].values
    y = df[target_col].values
    dx = df_tail[features].values
    dy = df_tail[target_col].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    scaler = MinMaxScaler()
    scaler1 = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    dx = scaler.transform(dx)
    y_train = scaler1.fit_transform(y_train.reshape(-1, 1))
    y_test = scaler1.transform(y_test.reshape(-1, 1))

    X_train_seq, _ = create_sequences(X_train, seq_length)
    y_train_seq, _ = create_sequences(y_train, seq_length)
    y_test_seq, _ = create_sequences(y_test, seq_length)
    X_test_seq, _ = create_sequences(X_test, seq_length)
    dx_seq, _ = create_sequences(dx, seq_length)

    model = Sequential()
    model.add(LSTM(75, activation='relu', input_shape=(seq_length, len(features))))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))

    opt = tf.keras.optimizers.Adam(learning_rate=0.005)

    model.compile(optimizer=opt, loss='mean_squared_error', metrics=['accuracy'])
    from tensorflow.keras.callbacks import EarlyStopping
    early_stop = EarlyStopping(monitor='val_accuracy', patience=50, restore_best_weights=True)
    history = model.fit(X_train_seq, y_train[seq_length:], epochs=500, batch_size=64, validation_split=0.3, callbacks=[early_stop], verbose=None)

    loss = model.evaluate(X_test_seq, y_test_seq, verbose=None)
    print(f'Test loss: {loss}')

    predictions = model.predict(dx_seq, verbose=None)
    predictions = scaler1.inverse_transform(predictions)

    predicted_prices = predictions.reshape(-1, 1)
    real_prices = dy[:-seq_length]
    days = list(range(1, dayspan - seq_length + 1))

    return predicted_prices, real_prices, days

def plotBefore(predicted_prices, real_prices, days):
    plt.figure(figsize=(12, 6))
    plt.plot(days, predicted_prices, label="Predicted Prices", marker='o', linestyle='-', color='b')
    plt.plot(days, real_prices, label="Real Prices", marker='x', linestyle='-', color='g')
    plt.xlabel("Days")
    plt.ylabel("Stock Price")
    plt.title("Predicted vs. Real Stock Prices")
    plt.legend()
    plt.grid(True)
    plt.show()

def joinSent(df, sent):
    import pandas as pd
    sen = []
    for i in range(len(sent)):
        a = pd.read_csv(sent[i] + '.csv')
        a.iloc[:, 1:] = (a.iloc[:, 1:] * 1000).round(3)
        sen.append(a)
    sen = pd.concat(sen, axis=0, ignore_index=True)
    sen = sen.sort_values(by='Date')
    sen.reset_index(drop=True, inplace=True)
    sen = sen.groupby('Date').mean().reset_index()
    sen.set_index('Date', inplace=True)
    df = pd.concat((sen, df), axis=1)
    df = df.ffill().fillna(method='backfill')
    return df

# Example usage
data = getData('TSLA', '2022-10-01', '2023-11-05')
sent = ['Oct2022', 'Nov2022', 'Dec2022', 'Jan2023', 'Feb2023', 'Mar2023', 'Apr2023', 'May2023', 'Jun2023', 'Jul2023', 'Ago2023', 'Sep2023', 'Oct2023', 'Nov2023']
data = joinSent(data, sent)

pred, real, days = predLSTM(data, 'Close', 30, 3)
plotBefore(pred, real, days)
